In [ ]:
# Our model use the same dataset and data splitting with Uni-Mol 
# Data is available at https://github.com/dptech-corp/Uni-Mol
import os
os.system('wget https://unimol.dp.tech/data/finetune/molecular_property_prediction.tar.gz')
os.system('tar xzvf molecular_property_prediction.tar.gz')

In [ ]:
# Convert the data from uni-Mol into the .csv file
import pandas as pd
import numpy as np
import lmdb
import os
import pickle

# 'HIV', 'BBBP', 'Tox21', 'SIDER', 'MUV', 'BACE', 'ClinTox', 'ToxCast', 'PCBA', 'ESOL', 'FreeSolv', 'Lipo', 'QM7', 'QM8', 'QM9'
name = 'ESOL'

name_uni_mol = str.lower(name)
if 'QM' in name:
    name_uni_mol = name_uni_mol + 'dft'
pos_smiles = None
print('Read and process the collected data...')
file = pd.read_csv('./' + name + '.csv', header=0)
title = file.columns.values
for i in range(len(title)):
    if title[i]  == 'smiles' or (name=='BACE' and title[i]  == 'mol'):
        pos_smiles = i
if pos_smiles is None:
    raise RuntimeError('Fail to find SMILES!!!')
    
molecules_list = file.values.tolist()
print('----------------------------------------')
print('example:', molecules_list[0],'...')

all_smiles = []
for i in range(len(molecules_list)):
    all_smiles.append(molecules_list[i][pos_smiles])
smiles_to_index = {all_smiles[i]:i for i in range(len(all_smiles))}

train = []
env = lmdb.open('./molecular_property_prediction/'+name_uni_mol+'/train.lmdb',subdir=False,readonly=True,lock=False,readahead=False,meminit=False,max_readers=256,)
txn = env.begin()
keys = list(txn.cursor().iternext(values=False))
for idx in keys:
    datapoint_pickled = txn.get(idx)
    data = pickle.loads(datapoint_pickled)
    train.append(data['smi'])

valid = []
env = lmdb.open('./molecular_property_prediction/'+name_uni_mol+'/valid.lmdb',subdir=False,readonly=True,lock=False,readahead=False,meminit=False,max_readers=256,)
txn = env.begin()
keys = list(txn.cursor().iternext(values=False))
for idx in keys:
    datapoint_pickled = txn.get(idx)
    data = pickle.loads(datapoint_pickled)
    valid.append(data['smi'])

test = []
env = lmdb.open('./molecular_property_prediction/'+name_uni_mol+'/test.lmdb',subdir=False,readonly=True,lock=False,readahead=False,meminit=False,max_readers=256,)
txn = env.begin()
keys = list(txn.cursor().iternext(values=False))
for idx in keys:
    datapoint_pickled = txn.get(idx)
    data = pickle.loads(datapoint_pickled)
    test.append(data['smi'])

# The return values are dc.data.Dataset objects so we need to extract the ids
print("Training set")
print(len(train))
print("Valid set")
print(len(valid))
print("Test set")
print(len(test))

if name in ['QM7']:
    molecules_list_new = [[len(train), len(valid)]]
else:
    molecules_list_new = [[len(train), len(valid), len(test)]]
for i in train:
    molecules_list_new.append(molecules_list[smiles_to_index[i]])
for i in valid:
    molecules_list_new.append(molecules_list[smiles_to_index[i]])
for i in test:
    molecules_list_new.append(molecules_list[smiles_to_index[i]])
            
file_output = pd.DataFrame(columns=None,data=molecules_list_new)
file_output.to_csv(name+'Scaffold.csv',header=list(file.columns.values),index = False)
